<a href="https://colab.research.google.com/github/Ahmed11Raza/Assignments-PIAIC-/blob/main/Tool_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Gemini API: Tool/Function calling with Python:

##Step by step Guide:-

**1. Function Description:**
You first define a function (or multiple functions) within your code. Each function serves a specific task, and you can provide a description of what each function does. For example, you could define a function for adding two numbers or fetching the current weather.

**2. Request with Description:**
When making a request to an AI model, you can include a description of what you want the AI to perform. The request includes not just the input data but also a detailed description of the task or the function you want to call.

**3. AI Identifies Matching Function:**
The AI model processes the request and matches the description to one of the defined functions. This is possible because the model has been trained to recognize patterns and match descriptions with appropriate functions.

**4. Function Name and Arguments:**
Once the model identifies the correct function, it will return the name of that function along with any required arguments. The returned function can then be called programmatically with those arguments to perform the task.

**5. Real-World Example:**
Imagine you're building a chatbot that helps users check the weather. You define a function called get_weather that takes a location as an argument and returns weather data. When the user asks the chatbot for the weather in a city, the model will:

Identify that the "*get_weather*" function matches the description of what the user is asking for.
Provide the function name (*get_weather*) along with the location as the argument.

# Install dependencies:

In [37]:
!pip install -U -q "google-generativeai>=0.7.2"  # Install the Python SDK

In [38]:
import google.generativeai as genai

# Set up your API key:
## To run the following cell, your API key must be stored it in a Colab Secret named GOOGLE_API_KEY. If you don't already have an API key, or you're not sure how to create a Colab Secret, see the Authentication quickstart for an example.

In [39]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

# Function calling basics

# Function calling in AI models allows you to define functions in your code and let the model choose and call the correct one based on a user’s request.

**Here’s how it works:**

Define Functions: You create functions with clear names, descriptions, and expected input types (like int, float, str, list, etc.).
Pass Functions to the Model: These functions are given to the model as "tools," which it can use when responding to requests.
Model Chooses the Right Function: Based on the user's input, the model looks at the function descriptions and parameter types to pick the correct function and provide the required arguments.
Function Call: The model sends back the function name and arguments, which your code then uses to call the function and get the result.
Allowed Parameter Types:
int, float, bool, str, list, dict (only these types are supported).
**Example**:

You define a function to add two numbers, and when a user asks for a sum, the model calls your function with the numbers.

In [40]:
def add(a: float, b: float):
    """returns a + b."""
    return a + b


def subtract(a: float, b: float):
    """returns a - b."""
    return a - b


def multiply(a: float, b: float):
    """returns a * b."""
    return a * b


def divide(a: float, b: float):
    """returns a / b."""
    return a / b


model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", tools=[add, subtract, multiply, divide]
)

model

genai.GenerativeModel(
    model_name='models/gemini-1.5-flash',
    generation_config={},
    safety_settings={},
    tools=<google.generativeai.types.content_types.FunctionLibrary object at 0x782432c601c0>,
    system_instruction=None,
    cached_content=None
)

#Automatic function calling:

# Function calls in multi-turn chats are designed to handle interactive, back-and-forth communication between the user and the AI model. They are especially useful when the AI needs to dynamically select and execute functions based on the evolving context of a conversation.

# Key Points:
**Natural Fit for Multi-Turn Chats:**

In a multi-turn chat, the user's request might involve multiple steps or require clarification (e.g., "Get the weather for Karachi," followed by "What about tomorrow?").
Function calls enable the model to respond appropriately by selecting and executing functions in real-time based on the ongoing conversation.
Using Python SDK's ChatSession:

The ChatSession class in the Python SDK manages the conversation history, so you don’t need to manually track past interactions.
The session ensures that the model has context from previous turns, allowing it to provide coherent and context-aware responses.
enable_automatic_function_calling:

By enabling the enable_automatic_function_calling parameter in the chat session, the SDK simplifies the process of function calling.
With this parameter, the model can:
Automatically determine which function to call based on the user's input.
Provide the necessary arguments for the function.
Integrate the function output into its response without requiring extra steps from the developer.
**Example Workflow:**
1. User Request:

User: "Get the weather for Karachi."
2. AI Response:

The model identifies a function (e.g., get_weather) and provides the argument (location = "Karachi").
3. Function Call:

The SDK automatically handles calling the get_weather function with the correct argument.
4. Result Integration:

The model receives the function's output and includes it in its response.
Model: "The weather in Karachi is sunny."
5. Next Turn:

 User: "What about tomorrow?"
The session maintains context, so the model knows the user is still asking about Karachi’s weather and can handle this seamlessly.
**Benefits:**

1) Simplified Development: You don’t need to manually track conversation history or map inputs to functions.

2) Dynamic and Interactive: Functions are selected and called dynamically based on the context of the conversation.

3) Coherent Conversations: The session maintains continuity, so users can have natural, multi-turn interactions.

In [41]:
chat = model.start_chat(enable_automatic_function_calling=True)

# When automatic function calling is enabled in the Python SDK's ChatSession, the send_message method does more than just generate text responses—it can also automatically identify and execute your predefined functions if the model determines one is needed to answer the user's query.

# This process integrates function calling seamlessly into the chat flow, making it appear as though the model is directly responding with the correct answer, even though it may have called a function behind the scenes to compute or retrieve the result.

# How It Works:

**1) Automatic Function Execution:**

When the user sends a message *(e.g., "What’s the weather in Karachi?")*, the send_message method:
Processes the user input.
      Determines if a function from the provided tools should be called based on the context and user intent.
If a function is needed, the SDK automatically executes it with the correct arguments.

**2) Combining Results into a Response:**

After the function is called, the SDK integrates the result into the AI's response as if it were part of a standard text reply.

**3) Simplified Development:**

You don’t need to write extra code to handle the function call manually—the SDK manages everything, from selecting the function to including the result in the reply.

In [42]:
response = chat.send_message(
    "I have 25 dogs, each has 4 legs, how many legs are there in total?"
)
response.text


'There are 100 legs in total.\n'

The ChatSession.history property in the Python SDK allows you to examine the entire flow of a conversation, showing how function calls are seamlessly integrated into the interaction between the user and the model.

# Here’s a breakdown of how it works:
**What ChatSession.history Tracks:**

**Conversation Turns:**
Each user input or model response is logged as a turn in the conversation.

**Content Details:** Each turn is represented by a genai.protos.Content object, which contains:
Role: Indicates who initiated the content:
"user": Messages sent by the user.
"model": Responses generated by the AI model.
Parts: The components of the message, which could include:
Text: Plain text messages.

**Function Call:**
 The model's request to execute a specific function.
Function Response: The result returned after the requested function is executed.

*Example:*
 Mittens Calculation
Let’s say the user asks a question that requires a function call to compute the result.

# Conversation Flow:
**User Input:**

1) The user asks: "I have 57 cats, each owns 44 mittens, how many mittens is that in total?"
This is logged in the conversation history as a "user" turn with plain text.

2)Model Function Call:

The model determines it needs a multiplication function to calculate the total mittens.
It logs a "model" turn with a FunctionCall part, specifying:
Function: multiply
Arguments: { "a": 57, "b": 44 }

3)Automatic Function Execution:

Since enable_automatic_function_calling is enabled, the ChatSession automatically executes the multiply function and calculates the result (57 × 44 = 2508).
A "user" turn with a FunctionResponse part is added to the history, showing the returned value: 2508.

**Model Response**:

The model uses the function result to generate its reply: *"The total number of mittens is 2508."*
This is logged as a "model" turn with plain text.

**Benefits of ChatSession.history:**

Full Transparency: You can see how each interaction and function call contributes to the final response.

**Debugging Made Easy:**
 By examining the sequence of turns, you can troubleshoot issues like incorrect function arguments or responses.
Traceable Conversations: The flow is stored chronologically, providing a clear timeline of the interaction.

In [43]:
for content in chat.history:
    print(content.role, "->", [type(part).to_dict(part) for part in content.parts])
    print("-" * 80)

user -> [{'text': 'I have 25 dogs, each has 4 legs, how many legs are there in total?'}]
--------------------------------------------------------------------------------
model -> [{'function_call': {'name': 'multiply', 'args': {'a': 25.0, 'b': 4.0}}}]
--------------------------------------------------------------------------------
user -> [{'function_response': {'name': 'multiply', 'response': {'result': 100.0}}}]
--------------------------------------------------------------------------------
model -> [{'text': 'There are 100 legs in total.\n'}]
--------------------------------------------------------------------------------


In [44]:
for content in chat.history:
    print(content.role, "->", [part.to_dict() if hasattr(part, 'to_dict') else str(part) for part in content.parts])
    print("=" * 80)  # Changed divider style


user -> ['text: "I have 25 dogs, each has 4 legs, how many legs are there in total?"\n']
model -> ['function_call {\n  name: "multiply"\n  args {\n    fields {\n      key: "b"\n      value {\n        number_value: 4\n      }\n    }\n    fields {\n      key: "a"\n      value {\n        number_value: 25\n      }\n    }\n  }\n}\n']
user -> ['function_response {\n  name: "multiply"\n  response {\n    fields {\n      key: "result"\n      value {\n        number_value: 100\n      }\n    }\n  }\n}\n']
model -> ['text: "There are 100 legs in total.\\n"\n']


# Manual function calling

For more control, you can process [`genai.protos.FunctionCall`](https://ai.google.dev/api/python/google/generativeai/protos/FunctionCall) requests from the model yourself. This would be the case if:

- You use a `ChatSession` with the default `enable_automatic_function_calling=False`.
- You use `GenerativeModel.generate_content` (and manage the chat history yourself).

The following example is a rough equivalent of the [function calling single-turn curl sample](https://ai.google.dev/docs/function_calling#function-calling-single-turn-curl-sample) in Python. It uses functions that return (mock) movie playtime information, possibly from a hypothetical API:

In [45]:
def find_cuisines(description: str, location: str = ""):
    """Find cuisines or dishes available in restaurants based on description, type, or popular keywords.

    Args:
        description: Any kind of description including cuisine type, popular dishes, or dietary preferences.
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
    """
    return ["Italian", "Mexican"]


def find_restaurants(location: str, cuisine: str = ""):
    """Find restaurants based on location and optionally filter by cuisine.

    Args:
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
        cuisine: Type of cuisine to filter restaurants by
    """
    return ["Pasta Palace", "Taco Tower"]


def get_reservation_times(location: str, restaurant: str, cuisine: str, date: str):
    """
    Find available reservation times for a specific restaurant.

    Args:
        location: The city and state, e.g. San Francisco, CA or a zip code e.g. 95616
        restaurant: Name of the restaurant
        cuisine: Type of cuisine served at the restaurant
        date: Date for requested reservation
    """
    return ["7:00 PM", "8:30 PM"]


# Explanation:

**Function 1:**
 **find_cuisines**

**Purpose:** Lists cuisines or dishes based on description or preferences.

**Arguments:**
**description:**
 Keywords like "spicy food" or "vegan options".

**location:** Optionally specify a location to narrow results.

Use a dictionary to make looking up functions by name easier later on. You can also use it to pass the array of functions to the `tools` parameter of `GenerativeModel`.

In [46]:
functions = {
    "find_cuisines": find_cuisines,
    "find_restaurants": find_restaurants,
    "get_reservation_times": get_reservation_times,
}

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=functions.values())


After using `generate_content()` to ask a question, the model requests a `function_call`:

In [47]:
response = model.generate_content(
    "Which restaurants in San Francisco serve Italian cuisine?"
)
response.candidates[0].content.parts


[function_call {
  name: "find_restaurants"
  args {
    fields {
      key: "location"
      value {
        string_value: "San Francisco"
      }
    }
    fields {
      key: "cuisine"
      value {
        string_value: "Italian"
      }
    }
  }
}
]

Instead of manually using if statements to call specific functions (e.g., find_restaurants, find_cuisines, etc.), you can leverage the functions dictionary to simplify the process. The dictionary allows you to dynamically call the appropriate function by its name, making the code cleaner and more maintainable.


Since this is not using a `ChatSession` with automatic function calling, you have to call the function yourself.

A very simple way to do this would be with `if` statements:

```python
if function_call.name == 'find_restaurants':
  find_restaurants(**function_call.args)
elif ...
```

However, since you already made the `functions` dictionary, this can be simplified to:

In [48]:
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    return functions[function_name](**function_args)


part = response.candidates[0].content.parts[0]

# Check if it's a function call; in real use you'd need to also handle text
# responses as you won't know what the model will respond with.
if part.function_call:
    result = call_function(part.function_call, functions)

print(result)

['Pasta Palace', 'Taco Tower']


# The final step involves continuing the conversation by passing the function response along with the message history to the model's generate_content() method. This allows the model to generate a cohesive and context-aware reply based on the earlier interaction.

In [49]:
from google.protobuf.struct_pb2 import Struct

# Simulated function result (ensure this matches the model's expectations)
result = {"restaurants": ["Pasta Palace", "Taco Tower"]}

# Put the result in a protobuf Struct
s = Struct()
s.update(result)

# Correct function response to match the function call
function_response = genai.protos.Part(
    function_response=genai.protos.FunctionResponse(name="find_restaurants", response=s)
)

# Build the message history
# Ensure the number of function call parts and function response parts is equal
messages = [
    {"role": "user", "parts": ["Which restaurants in San Francisco serve Italian cuisine?"]},
    {"role": "model", "parts": response.candidates[0].content.parts},  # Ensure this part aligns with the model's response
    {"role": "user", "parts": [function_response]},  # Correctly structured function response
]

# Generate the next response
response = model.generate_content(messages)
print(response.text)


Based on the available information, Pasta Palace and Taco Tower are in San Francisco and serve Italian food.  However, it's important to note that the response may be incomplete or inaccurate as the available tools may not have comprehensive data.



## Function calling chain

The model is not limited to one function call, it can chain them until it finds the right answer.

In [50]:
# Start the chat session with automatic function calling enabled
chat = model.start_chat(enable_automatic_function_calling=True)

# Send a message asking about restaurants serving Italian cuisine in San Francisco
response = chat.send_message(
    "Which restaurants in San Francisco serve Italian cuisine?"
)

# Iterate through the chat history to print the conversation
for content in chat.history:
    print(f"Role: {content.role} -> ", [type(part).to_dict(part) for part in content.parts])
    print("-" * 80)

# Optional: Handle the function response for restaurants
for content in chat.history:
    if content.role == "model" and hasattr(content, 'function_response'):
        print(f"Function Response: {content.function_response}")
        print("-" * 80)


Role: user ->  [{'text': 'Which restaurants in San Francisco serve Italian cuisine?'}]
--------------------------------------------------------------------------------
Role: model ->  [{'function_call': {'name': 'find_restaurants', 'args': {'location': 'San Francisco', 'cuisine': 'Italian'}}}]
--------------------------------------------------------------------------------
Role: user ->  [{'function_response': {'name': 'find_restaurants', 'response': {'result': ['Pasta Palace', 'Taco Tower']}}}]
--------------------------------------------------------------------------------
Role: model ->  [{'text': 'Pasta Palace and Taco Tower are two restaurants in San Francisco that serve Italian cuisine.  Note that this is based on the limited data available from the API.  There may be other Italian restaurants in San Francisco.\n'}]
--------------------------------------------------------------------------------


Here you can see that the model made three calls to answer your question and used the outputs of them in the subsequent calls and in the final answer.

## Parallel function calls

The Gemini API can call multiple functions in a single turn. This caters for scenarios where there are multiple function calls that can take place independently to complete a task.

First set the tools up. Unlike the Food Restuarant example above, these functions do not require input from each other to be called so they should be good candidates for parallel calling.

In [51]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

Now call the model with an instruction that could use all of the specified tools.

In [59]:
# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]
# Try this out with Pro and Flash...
model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Turn this place into a party!")

# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

power_disco_ball(power=True)
start_music(loud=True, energetic=True, bpm=120.0)
dim_lights(brightness=0.5)


Each of the printed results reflects a single function call that the model has requested. To send the results back, include the responses in the same order as they were requested.

In [60]:
# Simulate the responses from the specified tools.
responses = {
    "power_disco_ball": True,
    "start_music": "Never gonna give you up.",
    "dim_lights": True,
}

# Build the response parts.
response_parts = [
    genai.protos.Part(function_response=genai.protos.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

response = chat.send_message(response_parts)
print(response.text)

Party started!  Never gonna give you up is now playing. The disco ball is spinning and the lights are dimmed to 50% brightness.



## Next Steps
### Useful API references:

- The [genai.GenerativeModel](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/GenerativeModel.md) class
  - Its [GenerativeModel.generate_content](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/GenerativeModel.md#generate_content) method builds a [genai.protos.GenerateContentRequest](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/protos/GenerateContentRequest.md) behind the scenes.
    - The request's `.tools` field contains a list of 1 [genai.protos.Tool](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/protos/Tool.md) object.
    - The tool's `function_declarations` attribute contains a list of [FunctionDeclarations](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/protos/FunctionDeclaration.md) objects.
- The [response](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/protos/GenerateContentResponse.md) may contain a [genai.protos.FunctionCall](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/protos/FunctionCall.md), in `response.candidates[0].contents.parts[0]`.
- if `enable_automatic_function_calling` is set the [genai.ChatSession](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/ChatSession.md) executes the call, and sends back the [genai.protos.FunctionResponse](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/protos/FunctionResponse.md).
- In response to a [FunctionCall](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/protos/FunctionCall.md) the model always expects a [FunctionResponse](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/protos/FunctionResponse.md).
- If you reply manually using [chat.send_message](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/ChatSession.md#send_message) or [model.generate_content](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/GenerativeModel.md#generate_content) remember thart the API is stateless you have to send the whole conversation history (a list of [content](https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/protos/Content.md) objects), not just the last one containing the `FunctionResponse`.

### Related examples

Check those examples using function calling to give you more ideas on how to use that very useful feature:
* [Barista Bot](../examples/Agents_Function_Calling_Barista_Bot.ipynb), an agent to order coffee
* Using function calling to [re-rank seach results](../examples/Search_reranking_using_embeddings.ipynb)